# * VINSIGHT : Revenue
    Postpaid Revenue : TMH(Jan-Jun 2024) ใช้ data พี่เป้า(Definition เก่า ตัวเลขใกล้เคียงกัน)

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Import Transaction

In [2]:
''' Execute transaction '''


# Input parameter
v_start_date = 20240101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B0R000100' --Total Revenue
        , 'DB0R000100' --Total Revenue : DTAC
        , 'TB0R000100' --Total Revenue : TRUE
        , 'B0R000100GEO' --Total Revenue (Geo)
        , 'DB0R000100GEO' --Total Revenue (Geo) : DTAC
        , 'TB0R000100GEO' --Total Revenue (Geo) : TRUE

        , 'B0R000101' --Mobile Revenue
        , 'DB0R000101' --Mobile Revenue : DTAC
        , 'TB0R000101' --Mobile Revenue : TMH
        , 'B0R000101GEO' --Mobile Revenue (Geo)
        , 'DB0R000101GEO' --Mobile Revenue (Geo) : DTAC
        , 'TB0R000101GEO' --Mobile Revenue (Geo) : TMH
        
        , 'B1R000100' --Prepaid Revenue
        , 'DB1R000100' --Prepaid Revenue : DTAC
        , 'TB1R000100' --Prepaid Revenue : TMH
         
        , 'B2R000100' --Postpaid Revenue
        , 'B2R010100' --Postpaid Revenue B2C
        , 'B2R020100' --Postpaid Revenue B2B
        , 'DB2R000100' --Postpaid Revenue : DTAC
        , 'DB2R010100' --Postpaid Revenue B2C : DTAC
        , 'DB2R020100' --Postpaid Revenue B2B : DTAC
        , 'TB2R000100' --Postpaid Revenue : TMH
        , 'TB2R010100' --Postpaid Revenue B2C : TMH
        , 'TB2R020100' --Postpaid Revenue B2B : TMH
        
        , 'TB3R000100' --TOL Revenue
        
        , 'TB4R000100' --TVS Revenue
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    print(f'\nDataFrame: {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20240101

Data as of 2025-07-17, 10:08:20

DataFrame: 14638 rows, 16 columns


In [3]:
''' Automate Currently Period '''

curr_yr = chk_src_df['TM_KEY_YR'].max()
prev_yr = chk_src_df['TM_KEY_YR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_qtr = chk_src_df['TM_KEY_QTR'].max()
prev_qtr = chk_src_df['TM_KEY_QTR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = chk_src_df['TM_KEY_MTH'].drop_duplicates().sort_values().shift().max().astype(int)

curr_wk = chk_src_df['TM_KEY_WK'].max()
prev_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift().max().astype(int)
last_3_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift(3).max().astype(int)

# curr_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].max().astype(int)
# prev_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].drop_duplicates().shift().max().astype(int)

## Overview(Corp) by Period

In [4]:
''' Revenue(Corp) Yearly '''

v_metric_list = [
    'B0R000100' #Total Revenue
    , 'B1R000100' #Prepaid Revenue
    # , 'DB1R000100' #Prepaid Revenue : DTAC
    # , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R000100' #Postpaid Revenue
    # , 'DB2R000100' #Postpaid Revenue : DTAC
    # , 'TB2R000100' #Postpaid Revenue : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_corp_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'C']].copy()
rev_corp_yearly_df = rev_corp_yearly_df.loc[rev_corp_yearly_df['METRIC_CD'].isin(v_metric_list)]

rev_corp_yearly_df['TOTAL(Corp)'] = np.where(rev_corp_yearly_df['METRIC_CD']=='B0R000100', rev_corp_yearly_df['C'], 0)
rev_corp_yearly_df['PRE'] = np.where(rev_corp_yearly_df['METRIC_CD']=='B1R000100', rev_corp_yearly_df['C'], 0)
# rev_corp_yearly_df['PRE_T'] = np.where(rev_corp_yearly_df['METRIC_CD']=='TB1R000100', rev_corp_yearly_df['C'], 0)
# rev_corp_yearly_df['PRE_D'] = np.where(rev_corp_yearly_df['METRIC_CD']=='DB1R000100', rev_corp_yearly_df['C'], 0)
rev_corp_yearly_df['POST'] = np.where(rev_corp_yearly_df['METRIC_CD']=='B2R000100', rev_corp_yearly_df['C'], 0)
# rev_corp_yearly_df['POST_T'] = np.where(rev_corp_yearly_df['METRIC_CD']=='TB2R000100', rev_corp_yearly_df['C'], 0)
# rev_corp_yearly_df['POST_D'] = np.where(rev_corp_yearly_df['METRIC_CD']=='DB2R000100', rev_corp_yearly_df['C'], 0)
rev_corp_yearly_df['TOL'] = np.where(rev_corp_yearly_df['METRIC_CD']=='TB3R000100', rev_corp_yearly_df['C'], 0)
rev_corp_yearly_df['TVS'] = np.where(rev_corp_yearly_df['METRIC_CD']=='TB4R000100', rev_corp_yearly_df['C'], 0)

rev_corp_yearly_df = rev_corp_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL(Corp)':'sum', 'PRE':'sum', 'POST':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_corp_yearly_df['YoY'] = rev_corp_yearly_df['TOTAL(Corp)'] - rev_corp_yearly_df['TOTAL(Corp)'].shift()
rev_corp_yearly_df = rev_corp_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
rev_corp_yearly_df = rev_corp_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'YoY', 'TOTAL(Corp)', 'PRE', 'POST', 'TOL', 'TVS']]

rev_corp_yearly_df_display = rev_corp_yearly_df.copy()
rev_corp_yearly_df_display['ACTUAL_AS_OF'] = rev_corp_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = rev_corp_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_corp_yearly_df_display[col] = rev_corp_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
rev_corp_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,YoY,TOTAL(Corp),PRE,POST,TOL,TVS
0,2024,2025-07-17 05:19:06,20241231,0,"149,825,803,306","48,354,363,972","78,295,748,348","18,932,148,194","4,243,542,791"
1,2025,2025-07-17 05:19:06,20250715,"-68,827,781,950","80,998,021,356","26,859,839,801","41,620,780,251","10,437,182,717","1,923,713,589"


In [5]:
''' Revenue(Corp) Quarterly '''

v_metric_list = [
    'B0R000100' #Total Revenue
    , 'B1R000100' #Prepaid Revenue
    # , 'DB1R000100' #Prepaid Revenue : DTAC
    # , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R000100' #Postpaid Revenue
    # , 'DB2R000100' #Postpaid Revenue : DTAC
    # , 'TB2R000100' #Postpaid Revenue : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_corp_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'C']].copy()
rev_corp_quarterly_df = rev_corp_quarterly_df.loc[rev_corp_quarterly_df['METRIC_CD'].isin(v_metric_list)]

rev_corp_quarterly_df['TOTAL(Corp)'] = np.where(rev_corp_quarterly_df['METRIC_CD']=='B0R000100', rev_corp_quarterly_df['C'], 0)
rev_corp_quarterly_df['PRE'] = np.where(rev_corp_quarterly_df['METRIC_CD']=='B1R000100', rev_corp_quarterly_df['C'], 0)
# rev_corp_quarterly_df['PRE_T'] = np.where(rev_corp_quarterly_df['METRIC_CD']=='TB1R000100', rev_corp_quarterly_df['C'], 0)
# rev_corp_quarterly_df['PRE_D'] = np.where(rev_corp_quarterly_df['METRIC_CD']=='DB1R000100', rev_corp_quarterly_df['C'], 0)
rev_corp_quarterly_df['POST'] = np.where(rev_corp_quarterly_df['METRIC_CD']=='B2R000100', rev_corp_quarterly_df['C'], 0)
# rev_corp_quarterly_df['POST_T'] = np.where(rev_corp_quarterly_df['METRIC_CD']=='TB2R000100', rev_corp_quarterly_df['C'], 0)
# rev_corp_quarterly_df['POST_D'] = np.where(rev_corp_quarterly_df['METRIC_CD']=='DB2R000100', rev_corp_quarterly_df['C'], 0)
rev_corp_quarterly_df['TOL'] = np.where(rev_corp_quarterly_df['METRIC_CD']=='TB3R000100', rev_corp_quarterly_df['C'], 0)
rev_corp_quarterly_df['TVS'] = np.where(rev_corp_quarterly_df['METRIC_CD']=='TB4R000100', rev_corp_quarterly_df['C'], 0)

rev_corp_quarterly_df = rev_corp_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL(Corp)':'sum', 'PRE':'sum', 'POST':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_corp_quarterly_df['QoQ'] = rev_corp_quarterly_df['TOTAL(Corp)'] - rev_corp_quarterly_df['TOTAL(Corp)'].shift()
rev_corp_quarterly_df = rev_corp_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
rev_corp_quarterly_df = rev_corp_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'QoQ', 'TOTAL(Corp)', 'PRE', 'POST', 'TOL', 'TVS']]

rev_corp_quarterly_df_display = rev_corp_quarterly_df.copy()
rev_corp_quarterly_df_display['ACTUAL_AS_OF'] = rev_corp_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = rev_corp_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_corp_quarterly_df_display[col] = rev_corp_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
rev_corp_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,QoQ,TOTAL(Corp),PRE,POST,TOL,TVS
0,20241,2025-07-17 05:19:06,20240331,0,"37,260,216,440","11,843,094,082","19,753,202,663","4,602,316,250","1,061,603,446"
1,20242,2025-07-17 05:19:06,20240630,"-64,093,390","37,196,123,050","11,867,323,041","19,630,569,282","4,684,647,692","1,013,583,035"
2,20243,2025-07-17 05:19:06,20240930,"428,367,688","37,624,490,738","12,176,719,028","19,459,112,025","4,825,717,924","1,162,941,762"
3,20244,2025-07-17 05:19:06,20241231,"120,482,340","37,744,973,078","12,467,227,822","19,452,864,379","4,819,466,328","1,005,414,549"
4,20251,2025-07-17 05:19:06,20250331,"-221,910,719","37,523,062,359","12,392,416,294","19,368,310,563","4,823,173,375","939,162,126"
5,20252,2025-07-17 05:19:06,20250630,"-115,306,700","37,407,755,659","12,383,822,531","19,282,264,682","4,845,901,529","895,766,917"
6,20253,2025-07-17 05:19:06,20250715,"-31,340,552,320","6,067,203,339","2,083,600,976","2,970,205,006","768,107,813","88,784,546"


In [6]:
''' Revenue(Corp) Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
    'B0R000100' #Total Revenue
    , 'B1R000100' #Prepaid Revenue
    # , 'DB1R000100' #Prepaid Revenue : DTAC
    # , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R000100' #Postpaid Revenue
    # , 'DB2R000100' #Postpaid Revenue : DTAC
    # , 'TB2R000100' #Postpaid Revenue : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_corp_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'C']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr].copy()
rev_corp_monthly_df = rev_corp_monthly_df.loc[rev_corp_monthly_df['METRIC_CD'].isin(v_metric_list)]

rev_corp_monthly_df['TOTAL(Corp)'] = np.where(rev_corp_monthly_df['METRIC_CD']=='B0R000100', rev_corp_monthly_df['C'], 0)
rev_corp_monthly_df['PRE'] = np.where(rev_corp_monthly_df['METRIC_CD']=='B1R000100', rev_corp_monthly_df['C'], 0)
# rev_corp_monthly_df['PRE_T'] = np.where(rev_corp_monthly_df['METRIC_CD']=='TB1R000100', rev_corp_monthly_df['C'], 0)
# rev_corp_monthly_df['PRE_D'] = np.where(rev_corp_monthly_df['METRIC_CD']=='DB1R000100', rev_corp_monthly_df['C'], 0)
rev_corp_monthly_df['POST'] = np.where(rev_corp_monthly_df['METRIC_CD']=='B2R000100', rev_corp_monthly_df['C'], 0)
# rev_corp_monthly_df['POST_T'] = np.where(rev_corp_monthly_df['METRIC_CD']=='TB2R000100', rev_corp_monthly_df['C'], 0)
# rev_corp_monthly_df['POST_D'] = np.where(rev_corp_monthly_df['METRIC_CD']=='DB2R000100', rev_corp_monthly_df['C'], 0)
rev_corp_monthly_df['TOL'] = np.where(rev_corp_monthly_df['METRIC_CD']=='TB3R000100', rev_corp_monthly_df['C'], 0)
rev_corp_monthly_df['TVS'] = np.where(rev_corp_monthly_df['METRIC_CD']=='TB4R000100', rev_corp_monthly_df['C'], 0)

rev_corp_monthly_df = rev_corp_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL(Corp)':'sum', 'PRE':'sum', 'POST':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_corp_monthly_df['MoM'] = rev_corp_monthly_df['TOTAL(Corp)'] - rev_corp_monthly_df['TOTAL(Corp)'].shift()
rev_corp_monthly_df = rev_corp_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
rev_corp_monthly_df = rev_corp_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'MoM', 'TOTAL(Corp)', 'PRE', 'POST', 'TOL', 'TVS']]

rev_corp_monthly_df_display = rev_corp_monthly_df.copy()
rev_corp_monthly_df_display['ACTUAL_AS_OF'] = rev_corp_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = rev_corp_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_corp_monthly_df_display[col] = rev_corp_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
rev_corp_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,MoM,TOTAL(Corp),PRE,POST,TOL,TVS
0,202401,2025-07-17 05:19:06,20240131,0,"12,527,775,340","4,014,762,951","6,633,797,001","1,523,586,477","355,628,910"
1,202402,2025-07-17 05:19:06,20240229,"-279,732,061","12,248,043,279","3,796,987,160","6,572,934,091","1,530,089,455","348,032,572"
2,202403,2025-07-17 05:19:06,20240331,"236,354,542","12,484,397,821","4,031,343,970","6,546,471,571","1,548,640,318","357,941,963"
3,202404,2025-07-17 05:19:06,20240430,"-125,805,023","12,358,592,798","3,884,370,013","6,572,828,730","1,554,149,465","347,244,590"
4,202405,2025-07-17 05:19:06,20240531,"107,213,556","12,465,806,354","4,046,469,103","6,520,016,738","1,561,523,330","337,797,184"
5,202406,2025-07-17 05:19:06,20240630,"-94,082,457","12,371,723,897","3,936,483,925","6,537,723,813","1,568,974,898","328,541,261"
6,202407,2025-07-17 05:19:06,20240731,"118,155,942","12,489,879,839","4,077,341,056","6,484,830,524","1,585,220,761","342,487,499"
7,202408,2025-07-17 05:19:06,20240831,"134,708,079","12,624,587,918","4,114,136,410","6,483,049,467","1,598,755,814","428,646,226"
8,202409,2025-07-17 05:19:06,20240930,"-114,564,937","12,510,022,981","3,985,241,562","6,491,232,034","1,641,741,349","391,808,037"
9,202410,2025-07-17 05:19:06,20241031,"59,487,568","12,569,510,549","4,131,927,606","6,485,731,903","1,614,615,686","337,235,354"


In [7]:
''' Revenue(Corp) Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
    'B0R000100' #Total Revenue
    , 'B1R000100' #Prepaid Revenue
    # , 'DB1R000100' #Prepaid Revenue : DTAC
    # , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R000100' #Postpaid Revenue
    # , 'DB2R000100' #Postpaid Revenue : DTAC
    # , 'TB2R000100' #Postpaid Revenue : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_corp_weekly_df = chk_src_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'C']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
rev_corp_weekly_df = rev_corp_weekly_df.loc[rev_corp_weekly_df['METRIC_CD'].isin(v_metric_list)]

rev_corp_weekly_df['TOTAL(Corp)'] = np.where(rev_corp_weekly_df['METRIC_CD']=='B0R000100', rev_corp_weekly_df['C'], 0)
rev_corp_weekly_df['PRE'] = np.where(rev_corp_weekly_df['METRIC_CD']=='B1R000100', rev_corp_weekly_df['C'], 0)
# rev_corp_weekly_df['PRE_T'] = np.where(rev_corp_weekly_df['METRIC_CD']=='TB1R000100', rev_corp_weekly_df['C'], 0)
# rev_corp_weekly_df['PRE_D'] = np.where(rev_corp_weekly_df['METRIC_CD']=='DB1R000100', rev_corp_weekly_df['C'], 0)
rev_corp_weekly_df['POST'] = np.where(rev_corp_weekly_df['METRIC_CD']=='B2R000100', rev_corp_weekly_df['C'], 0)
# rev_corp_weekly_df['POST_T'] = np.where(rev_corp_weekly_df['METRIC_CD']=='TB2R000100', rev_corp_weekly_df['C'], 0)
# rev_corp_weekly_df['POST_D'] = np.where(rev_corp_weekly_df['METRIC_CD']=='DB2R000100', rev_corp_weekly_df['C'], 0)
rev_corp_weekly_df['TOL'] = np.where(rev_corp_weekly_df['METRIC_CD']=='TB3R000100', rev_corp_weekly_df['C'], 0)
rev_corp_weekly_df['TVS'] = np.where(rev_corp_weekly_df['METRIC_CD']=='TB4R000100', rev_corp_weekly_df['C'], 0)

rev_corp_weekly_df = rev_corp_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL(Corp)':'sum', 'PRE':'sum', 'POST':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_corp_weekly_df['WoW'] = rev_corp_weekly_df['TOTAL(Corp)'] - rev_corp_weekly_df['TOTAL(Corp)'].shift()
rev_corp_weekly_df = rev_corp_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
rev_corp_weekly_df = rev_corp_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'WoW', 'TOTAL(Corp)', 'PRE', 'POST', 'TOL', 'TVS']]

rev_corp_weekly_df_display = rev_corp_weekly_df.copy()
rev_corp_weekly_df_display['ACTUAL_AS_OF'] = rev_corp_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = rev_corp_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_corp_weekly_df_display[col] = rev_corp_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
rev_corp_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,WoW,TOTAL(Corp),PRE,POST,TOL,TVS
0,2025022,2025-07-17 05:19:06,20250601,0,"173,158,839","165,369,424","7,789,415",0,0
1,2025023,2025-07-17 05:19:06,20250608,"2,877,642,426","3,050,801,265","994,546,788","1,645,661,470","367,645,423","42,947,584"
2,2025024,2025-07-17 05:19:06,20250615,"-269,704,746","2,781,096,519","872,935,959","1,454,821,044","404,636,671","48,702,846"
3,2025025,2025-07-17 05:19:06,20250622,"325,158,950","3,106,255,469","880,788,388","1,662,242,728","502,657,725","60,566,629"
4,2025026,2025-07-17 05:19:06,20250629,"-266,967,779","2,839,287,691","923,859,461","1,540,070,632","335,922,404","39,435,194"
5,2025027,2025-07-17 05:19:06,20250706,"264,144,511","3,103,432,201","1,096,689,416","1,514,956,573","368,079,619","123,706,593"
6,2025028,2025-07-17 05:19:06,20250713,"-264,235,617","2,839,196,584","921,911,827","1,469,591,862","400,028,194","47,664,701"
7,2025029,2025-07-17 05:19:06,20250715,"-2,427,679,361","411,517,224","255,012,225",0,0,0


In [8]:
''' Revenue(Corp) Daily '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

v_metric_list = [
    'B0R000100' #Total Revenue
    , 'B1R000100' #Prepaid Revenue
    # , 'DB1R000100' #Prepaid Revenue : DTAC
    # , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R000100' #Postpaid Revenue
    # , 'DB2R000100' #Postpaid Revenue : DTAC
    # , 'TB2R000100' #Postpaid Revenue : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_corp_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
rev_corp_daily_df = rev_corp_daily_df.loc[rev_corp_daily_df['METRIC_CD'].isin(v_metric_list)]

rev_corp_daily_df['TOTAL(Corp)'] = np.where(rev_corp_daily_df['METRIC_CD']=='B0R000100', rev_corp_daily_df['C'], 0)
rev_corp_daily_df['PRE'] = np.where(rev_corp_daily_df['METRIC_CD']=='B1R000100', rev_corp_daily_df['C'], 0)
# rev_corp_daily_df['PRE_T'] = np.where(rev_corp_daily_df['METRIC_CD']=='TB1R000100', rev_corp_daily_df['C'], 0)
# rev_corp_daily_df['PRE_D'] = np.where(rev_corp_daily_df['METRIC_CD']=='DB1R000100', rev_corp_daily_df['C'], 0)
rev_corp_daily_df['POST'] = np.where(rev_corp_daily_df['METRIC_CD']=='B2R000100', rev_corp_daily_df['C'], 0)
# rev_corp_daily_df['POST_T'] = np.where(rev_corp_daily_df['METRIC_CD']=='TB2R000100', rev_corp_daily_df['C'], 0)
# rev_corp_daily_df['POST_D'] = np.where(rev_corp_daily_df['METRIC_CD']=='DB2R000100', rev_corp_daily_df['C'], 0)
rev_corp_daily_df['TOL'] = np.where(rev_corp_daily_df['METRIC_CD']=='TB3R000100', rev_corp_daily_df['C'], 0)
rev_corp_daily_df['TVS'] = np.where(rev_corp_daily_df['METRIC_CD']=='TB4R000100', rev_corp_daily_df['C'], 0)

rev_corp_daily_df = rev_corp_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'TOTAL(Corp)':'sum', 'PRE':'sum', 'POST':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_corp_daily_df['DoD'] = rev_corp_daily_df['TOTAL(Corp)'] - rev_corp_daily_df['TOTAL(Corp)'].shift()
rev_corp_daily_df = rev_corp_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
rev_corp_daily_df = rev_corp_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'DoD', 'TOTAL(Corp)', 'PRE', 'POST', 'TOL', 'TVS']]

rev_corp_daily_df_display = rev_corp_daily_df.copy()
mod_col_list = rev_corp_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_corp_daily_df_display[col] = rev_corp_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
rev_corp_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,DoD,TOTAL(Corp),PRE,POST,TOL,TVS
0,202507,20250716,2025-07-17 05:19:06,"-194,181,854",0,0,0,0,0
1,202507,20250715,2025-07-17 05:19:06,"-23,153,515","194,181,854","116,903,928",0,0,0
2,202507,20250714,2025-07-17 05:19:06,"-566,433,433","217,335,369","138,108,297",0,0,0
3,202507,20250713,2025-07-17 05:19:06,"581,903,631","783,768,802","120,216,405","474,275,245","169,019,668","20,257,484"
4,202507,20250712,2025-07-17 05:19:06,"-9,176,681","201,865,172","121,447,614","80,417,557",0,0
5,202507,20250711,2025-07-17 05:19:06,"-758,176,090","211,041,853","127,617,648","83,424,205",0,0
6,202507,20250710,2025-07-17 05:19:06,"744,032,408","969,217,943","137,200,716","573,601,485","231,008,526","27,407,217"
7,202507,20250709,2025-07-17 05:19:06,"6,899,835","225,185,535","139,304,514","85,881,021",0,0
8,202507,20250708,2025-07-17 05:19:06,"-11,545,880","218,285,700","133,551,578","84,734,122",0,0
9,202507,20250707,2025-07-17 05:19:06,"-454,148,467","229,831,580","142,573,352","87,258,228",0,0


## Overview(Geo) by Period

In [9]:
''' Revenue(Geo) Yearly '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R010100' #Postpaid Revenue B2C
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
rev_geo_yearly_df = rev_geo_yearly_df.loc[rev_geo_yearly_df['METRIC_CD'].isin(v_metric_list)]

rev_geo_yearly_df['TOTAL(Geo)'] = np.where(rev_geo_yearly_df['METRIC_CD']=='B0R000100GEO', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['PRE'] = np.where(rev_geo_yearly_df['METRIC_CD']=='B1R000100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['PRE_T'] = np.where(rev_geo_yearly_df['METRIC_CD']=='TB1R000100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['PRE_D'] = np.where(rev_geo_yearly_df['METRIC_CD']=='DB1R000100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['POST_B2C'] = np.where(rev_geo_yearly_df['METRIC_CD']=='B2R010100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['POST_B2C_T'] = np.where(rev_geo_yearly_df['METRIC_CD']=='TB2R010100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['POST_B2C_D'] = np.where(rev_geo_yearly_df['METRIC_CD']=='DB2R010100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['TOL'] = np.where(rev_geo_yearly_df['METRIC_CD']=='TB3R000100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['TVS'] = np.where(rev_geo_yearly_df['METRIC_CD']=='TB4R000100', rev_geo_yearly_df['P'], 0)

rev_geo_yearly_df = rev_geo_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_yearly_df['YoY'] = rev_geo_yearly_df['TOTAL(Geo)'] - rev_geo_yearly_df['TOTAL(Geo)'].shift()
rev_geo_yearly_df = rev_geo_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
rev_geo_yearly_df = rev_geo_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'YoY', 'TOTAL(Geo)', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL', 'TVS']]

rev_geo_yearly_df_display = rev_geo_yearly_df.copy()
rev_geo_yearly_df_display['ACTUAL_AS_OF'] = rev_geo_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = rev_geo_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_geo_yearly_df_display[col] = rev_geo_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
rev_geo_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,YoY,TOTAL(Geo),PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL,TVS
0,2024,2025-07-17 05:19:06,20241231,0,"142,125,302,342","48,319,930,436","27,043,194,488","21,276,735,948","70,959,878,334","39,407,414,546","31,552,463,789","18,827,249,425","4,018,244,147"
1,2025,2025-07-17 05:19:06,20250715,"-65,257,010,499","76,868,291,843","26,689,965,552","15,372,835,266","11,372,439,607","37,675,575,817","21,530,677,350","16,295,169,626","10,408,786,318","1,826,875,011"


In [10]:
''' Revenue(Geo) Quarterly '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R010100' #Postpaid Revenue B2C
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
rev_geo_quarterly_df = rev_geo_quarterly_df.loc[rev_geo_quarterly_df['METRIC_CD'].isin(v_metric_list)]

rev_geo_quarterly_df['TOTAL(Geo)'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='B0R000100GEO', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['PRE'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='B1R000100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['PRE_T'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='TB1R000100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['PRE_D'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='DB1R000100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['POST_B2C'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='B2R010100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['POST_B2C_T'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='TB2R010100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['POST_B2C_D'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='DB2R010100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['TOL'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='TB3R000100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['TVS'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='TB4R000100', rev_geo_quarterly_df['P'], 0)

rev_geo_quarterly_df = rev_geo_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_quarterly_df['QoQ'] = rev_geo_quarterly_df['TOTAL(Geo)'] - rev_geo_quarterly_df['TOTAL(Geo)'].shift()
rev_geo_quarterly_df = rev_geo_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
rev_geo_quarterly_df = rev_geo_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'QoQ', 'TOTAL(Geo)', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL', 'TVS']]

rev_geo_quarterly_df_display = rev_geo_quarterly_df.copy()
rev_geo_quarterly_df_display['ACTUAL_AS_OF'] = rev_geo_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = rev_geo_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_geo_quarterly_df_display[col] = rev_geo_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
rev_geo_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,QoQ,TOTAL(Geo),PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL,TVS
0,20241,2025-07-17 05:19:06,20240331,0,"35,361,933,475","11,842,176,786","6,519,610,988","5,322,565,798","17,936,624,018","9,850,451,722","8,086,172,296","4,569,920,004","1,013,212,668"
1,20242,2025-07-17 05:19:06,20240630,"-77,400,943","35,284,532,532","11,858,030,293","6,615,472,616","5,242,557,676","17,792,137,434","9,838,450,875","7,953,686,559","4,665,781,597","968,583,208"
2,20243,2025-07-17 05:19:06,20240930,"422,283,505","35,706,816,037","12,164,985,004","6,830,749,822","5,334,235,182","17,628,033,120","9,834,028,205","7,794,004,915","4,803,785,483","1,110,012,430"
3,20244,2025-07-17 05:19:06,20241231,"65,204,262","35,772,020,298","12,454,738,354","7,077,361,062","5,377,377,292","17,603,083,763","9,884,483,743","7,718,600,020","4,787,762,341","926,435,841"
4,20251,2025-07-17 05:19:06,20250331,"-122,576,840","35,649,443,458","12,375,356,967","7,111,077,784","5,264,279,183","17,580,761,024","9,989,547,936","7,591,213,088","4,810,187,607","883,137,861"
5,20252,2025-07-17 05:19:06,20250630,"-120,733,603","35,528,709,855","12,349,913,007","7,120,338,187","5,229,574,820","17,484,715,551","9,954,637,500","7,530,078,051","4,833,931,536","860,149,762"
6,20253,2025-07-17 05:19:06,20250715,"-29,838,571,325","5,690,138,530","1,964,695,579","1,141,419,296","878,585,604","2,610,099,241","1,586,491,913","1,173,878,487","764,667,175","83,587,388"


In [11]:
''' Revenue(Geo) Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R010100' #Postpaid Revenue B2C
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr].copy()
rev_geo_monthly_df = rev_geo_monthly_df.loc[rev_geo_monthly_df['METRIC_CD'].isin(v_metric_list)]

rev_geo_monthly_df['TOTAL(Geo)'] = np.where(rev_geo_monthly_df['METRIC_CD']=='B0R000100GEO', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['PRE'] = np.where(rev_geo_monthly_df['METRIC_CD']=='B1R000100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['PRE_T'] = np.where(rev_geo_monthly_df['METRIC_CD']=='TB1R000100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['PRE_D'] = np.where(rev_geo_monthly_df['METRIC_CD']=='DB1R000100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['POST_B2C'] = np.where(rev_geo_monthly_df['METRIC_CD']=='B2R010100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['POST_B2C_T'] = np.where(rev_geo_monthly_df['METRIC_CD']=='TB2R010100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['POST_B2C_D'] = np.where(rev_geo_monthly_df['METRIC_CD']=='DB2R010100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['TOL'] = np.where(rev_geo_monthly_df['METRIC_CD']=='TB3R000100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['TVS'] = np.where(rev_geo_monthly_df['METRIC_CD']=='TB4R000100', rev_geo_monthly_df['P'], 0)

rev_geo_monthly_df = rev_geo_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_monthly_df['MoM'] = rev_geo_monthly_df['TOTAL(Geo)'] - rev_geo_monthly_df['TOTAL(Geo)'].shift()
rev_geo_monthly_df = rev_geo_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
rev_geo_monthly_df = rev_geo_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'MoM', 'TOTAL(Geo)', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL', 'TVS']]

rev_geo_monthly_df_display = rev_geo_monthly_df.copy()
rev_geo_monthly_df_display['ACTUAL_AS_OF'] = rev_geo_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = rev_geo_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_geo_monthly_df_display[col] = rev_geo_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
rev_geo_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,MoM,TOTAL(Geo),PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL,TVS
0,202401,2025-07-17 05:19:06,20240131,0,"11,881,730,399","4,014,674,524","2,211,126,641","1,803,547,883","6,020,820,300","3,311,282,032","2,709,538,268","1,507,069,397","339,166,179"
1,202402,2025-07-17 05:19:06,20240229,"-255,166,666","11,626,563,733","3,796,713,842","2,082,104,053","1,714,609,789","5,974,961,131","3,285,072,468","2,689,888,663","1,523,039,176","331,849,584"
2,202403,2025-07-17 05:19:06,20240331,"227,075,609","11,853,639,343","4,030,788,420","2,226,380,294","1,804,408,125","5,940,842,587","3,254,097,222","2,686,745,365","1,539,811,431","342,196,905"
3,202404,2025-07-17 05:19:06,20240430,"-130,159,939","11,723,479,404","3,883,185,086","2,140,030,223","1,743,154,862","5,961,586,281","3,280,178,978","2,681,407,303","1,546,917,278","331,790,759"
4,202405,2025-07-17 05:19:06,20240531,"118,388,442","11,841,867,846","4,042,374,933","2,269,268,086","1,773,106,846","5,921,618,672","3,273,314,626","2,648,304,046","1,554,971,583","322,902,658"
5,202406,2025-07-17 05:19:06,20240630,"-122,682,564","11,719,185,282","3,932,470,274","2,206,174,307","1,726,295,968","5,908,932,481","3,284,957,272","2,623,975,210","1,563,892,736","313,889,791"
6,202407,2025-07-17 05:19:06,20240731,"138,027,673","11,857,212,955","4,072,664,861","2,272,833,725","1,799,831,136","5,878,727,794","3,269,232,707","2,609,495,087","1,577,812,706","328,007,594"
7,202408,2025-07-17 05:19:06,20240831,"135,679,548","11,992,892,504","4,109,958,930","2,312,752,271","1,797,206,658","5,880,942,518","3,283,314,789","2,597,627,729","1,592,482,287","409,508,769"
8,202409,2025-07-17 05:19:06,20240930,"-136,181,926","11,856,710,578","3,982,361,213","2,245,163,825","1,737,197,388","5,868,362,808","3,281,480,710","2,586,882,098","1,633,490,490","372,496,067"
9,202410,2025-07-17 05:19:06,20241031,"43,673,946","11,900,384,524","4,128,174,548","2,345,529,988","1,782,644,560","5,862,510,509","3,272,186,553","2,590,323,955","1,598,256,224","311,443,243"


In [12]:
''' Revenue(Geo) Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R010100' #Postpaid Revenue B2C
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_weekly_df = chk_src_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
rev_geo_weekly_df = rev_geo_weekly_df.loc[rev_geo_weekly_df['METRIC_CD'].isin(v_metric_list)]

rev_geo_weekly_df['TOTAL(Geo)'] = np.where(rev_geo_weekly_df['METRIC_CD']=='B0R000100GEO', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['PRE'] = np.where(rev_geo_weekly_df['METRIC_CD']=='B1R000100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['PRE_T'] = np.where(rev_geo_weekly_df['METRIC_CD']=='TB1R000100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['PRE_D'] = np.where(rev_geo_weekly_df['METRIC_CD']=='DB1R000100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['POST_B2C'] = np.where(rev_geo_weekly_df['METRIC_CD']=='B2R010100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['POST_B2C_T'] = np.where(rev_geo_weekly_df['METRIC_CD']=='TB2R010100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['POST_B2C_D'] = np.where(rev_geo_weekly_df['METRIC_CD']=='DB2R010100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['TOL'] = np.where(rev_geo_weekly_df['METRIC_CD']=='TB3R000100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['TVS'] = np.where(rev_geo_weekly_df['METRIC_CD']=='TB4R000100', rev_geo_weekly_df['P'], 0)

rev_geo_weekly_df = rev_geo_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_weekly_df['WoW'] = rev_geo_weekly_df['TOTAL(Geo)'] - rev_geo_weekly_df['TOTAL(Geo)'].shift()
rev_geo_weekly_df = rev_geo_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
rev_geo_weekly_df = rev_geo_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'WoW', 'TOTAL(Geo)', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL', 'TVS']]

rev_geo_weekly_df_display = rev_geo_weekly_df.copy()
rev_geo_weekly_df_display['ACTUAL_AS_OF'] = rev_geo_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = rev_geo_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_geo_weekly_df_display[col] = rev_geo_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
rev_geo_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,WoW,TOTAL(Geo),PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL,TVS
0,2025022,2025-07-17 05:19:06,20250601,0,"169,983,041","165,152,372","94,695,780","70,456,592","4,830,669",0,"4,830,669",0,0
1,2025023,2025-07-17 05:19:06,20250608,"2,586,852,728","2,756,835,769","993,554,639","573,236,388","420,318,251","1,356,244,391","747,822,758","608,421,633","366,087,708","40,949,031"
2,2025024,2025-07-17 05:19:06,20250615,"-60,924,297","2,695,911,472","871,828,576","499,839,258","371,989,318","1,373,464,775","809,122,428","564,342,346","403,868,044","46,750,078"
3,2025025,2025-07-17 05:19:06,20250622,"326,850,918","3,022,762,390","877,080,187","489,184,615","387,895,572","1,584,885,025","1,037,219,331","547,665,694","501,711,711","59,085,467"
4,2025026,2025-07-17 05:19:06,20250629,"-323,867,414","2,698,894,975","919,142,945","529,613,518","389,529,427","1,405,714,437","643,927,416","761,787,021","335,311,493","38,726,100"
5,2025027,2025-07-17 05:19:06,20250706,"98,303,642","2,797,198,617","1,095,420,861","634,169,167","461,251,694","1,216,835,738","767,942,598","448,893,140","366,550,576","118,391,442"
6,2025028,2025-07-17 05:19:06,20250713,"-35,044,997","2,762,153,620","921,089,983","535,893,455","385,196,528","1,398,076,596","818,549,315","579,527,280","398,116,600","44,870,442"
7,2025029,2025-07-17 05:19:06,20250715,"-2,357,100,917","405,052,703","137,963,558","84,749,210","108,523,668",0,0,"150,271,159",0,0


In [13]:
''' Revenue(Geo) Daily '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R010100' #Postpaid Revenue B2C
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
rev_geo_daily_df = rev_geo_daily_df.loc[rev_geo_daily_df['METRIC_CD'].isin(v_metric_list)]

rev_geo_daily_df['TOTAL(Geo)'] = np.where(rev_geo_daily_df['METRIC_CD']=='B0R000100GEO', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['PRE'] = np.where(rev_geo_daily_df['METRIC_CD']=='B1R000100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['PRE_T'] = np.where(rev_geo_daily_df['METRIC_CD']=='TB1R000100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['PRE_D'] = np.where(rev_geo_daily_df['METRIC_CD']=='DB1R000100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['POST_B2C'] = np.where(rev_geo_daily_df['METRIC_CD']=='B2R010100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['POST_B2C_T'] = np.where(rev_geo_daily_df['METRIC_CD']=='TB2R010100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['POST_B2C_D'] = np.where(rev_geo_daily_df['METRIC_CD']=='DB2R010100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['TOL'] = np.where(rev_geo_daily_df['METRIC_CD']=='TB3R000100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['TVS'] = np.where(rev_geo_daily_df['METRIC_CD']=='TB4R000100', rev_geo_daily_df['P'], 0)

rev_geo_daily_df = rev_geo_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_daily_df['DoD'] = rev_geo_daily_df['TOTAL(Geo)'] - rev_geo_daily_df['TOTAL(Geo)'].shift()
rev_geo_daily_df = rev_geo_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
rev_geo_daily_df = rev_geo_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'DoD', 'TOTAL(Geo)', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL', 'TVS']]

rev_geo_daily_df_display = rev_geo_daily_df.copy()
mod_col_list = rev_geo_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_geo_daily_df_display[col] = rev_geo_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
rev_geo_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,DoD,TOTAL(Geo),PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL,TVS
0,202507,20250716,2025-07-17 05:19:06,"-191,770,529",0,0,0,0,0,0,0,0,0
1,202507,20250715,2025-07-17 05:19:06,"-21,511,645","191,770,529",0,0,"55,309,321",0,0,"74,952,542",0,0
2,202507,20250714,2025-07-17 05:19:06,"-549,164,982","213,282,174","137,963,558","84,749,210","53,214,347",0,0,"75,318,617",0,0
3,202507,20250713,2025-07-17 05:19:06,"561,363,839","762,447,156","120,082,954","68,692,576","51,390,378","455,883,219","375,592,889","80,290,331","167,587,317","18,893,666"
4,202507,20250712,2025-07-17 05:19:06,"-8,027,327","201,083,318","121,330,700","70,625,088","50,705,612","79,752,618",0,"79,752,618",0,0
5,202507,20250711,2025-07-17 05:19:06,"-714,448,785","209,110,645","127,517,748","74,018,797","53,498,951","81,592,897",0,"81,592,897",0,0
6,202507,20250710,2025-07-17 05:19:06,"700,603,640","923,559,430","137,099,469","80,386,582","56,712,887","529,953,902","442,956,427","86,997,475","230,529,283","25,976,776"
7,202507,20250709,2025-07-17 05:19:06,"6,595,468","222,955,789","139,179,457","82,049,956","57,129,501","83,776,332",0,"83,776,332",0,0
8,202507,20250708,2025-07-17 05:19:06,"-10,276,640","216,360,321","133,438,528","77,052,729","56,385,800","82,921,793",0,"82,921,793",0,0
9,202507,20250707,2025-07-17 05:19:06,"-447,878,111","226,636,961","142,441,127","83,067,727","59,373,400","84,195,834",0,"84,195,834",0,0


## Products Summary

### Prep Monthly Data

In [14]:
''' Monthly Summary '''

monthly_df = chk_src_df.copy()
monthly_df = monthly_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
monthly_df = monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
monthly_df['DIFF (C-P)'] = monthly_df['C'] - monthly_df['P']

monthly_df_display = monthly_df.copy()
mod_col_list = monthly_df_display.iloc[:, 5:].columns.tolist()
for col in mod_col_list:
    monthly_df_display[col] = monthly_df_display[col].apply(lambda x: format(x, ',.0f'))

# monthly_df_display
# monthly_df_display.loc[monthly_df_display['TM_KEY_MTH']==202501]

In [15]:
''' Parameter '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth
# v_tm_key_mth = 202505

print(f'v_tm_key_mth: {v_tm_key_mth}')

v_tm_key_mth: 202506


### All Service

In [16]:
''' All Service (Corp) '''

v_product_grp = 'All Services'

all_service_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp].copy()
all_service_df = all_service_df.loc[~all_service_df['METRIC_CD'].str.contains('GEO')]
all_service_df = all_service_df.loc[all_service_df['TM_KEY_MTH']==v_tm_key_mth]
all_service_df = all_service_df.reset_index(drop=True)
all_service_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202506,All Services,B0R000100,Total Revenue,2025-07-17 02:28:35,"12,237,542,453","12,213,630,387",0,0,0,"23,912,066"
1,202506,All Services,DB0R000100,Total Revenue : DTAC,2025-07-17 02:28:35,"4,402,672,131","4,402,676,634",0,0,0,"-4,503"
2,202506,All Services,TB0R000100,Total Revenue : TRUE,2025-07-17 02:28:35,"7,834,870,322","7,810,953,753",0,0,0,"23,916,569"


In [17]:
''' All Service (Geo) '''

v_product_grp = 'All Services'

all_service_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp].copy()
all_service_geo_df = all_service_geo_df.loc[all_service_geo_df['METRIC_CD'].str.contains('GEO')]
all_service_geo_df = all_service_geo_df.loc[all_service_geo_df['TM_KEY_MTH']==v_tm_key_mth]
all_service_geo_df = all_service_geo_df.reset_index(drop=True)
all_service_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202506,All Services,B0R000100GEO,Total Revenue (Geo),2025-07-17 05:19:06,0,"11,618,654,057","11,618,439,206","11,616,988,460","11,616,885,626","-11,618,654,057"
1,202506,All Services,DB0R000100GEO,Total Revenue (Geo) : DTAC,2025-07-17 05:19:06,0,"4,208,435,900","4,208,433,303","4,208,364,673","4,208,364,673","-4,208,435,900"
2,202506,All Services,TB0R000100GEO,Total Revenue (Geo) : TRUE,2025-07-17 05:19:06,0,"7,410,218,157","7,410,005,903","7,408,623,787","7,408,520,953","-7,410,218,157"


### Mobile

In [18]:
''' Mobile (Corp) '''

v_product_grp = 'Mobile'

mobile_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
mobile_df = mobile_df.loc[~mobile_df['METRIC_CD'].str.contains('GEO')]
mobile_df = mobile_df.loc[mobile_df['TM_KEY_MTH']==v_tm_key_mth]
mobile_df = mobile_df.reset_index(drop=True)
mobile_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202506,Mobile,B0R000101,Mobile Revenue,2025-07-17 02:28:35,"10,352,441,230","10,341,466,260",0,0,0,"10,974,970"
1,202506,Mobile,DB0R000101,Mobile Revenue : DTAC,2025-07-17 02:28:35,"4,402,672,131","4,402,676,634",0,0,0,"-4,503"
2,202506,Mobile,TB0R000101,Mobile Revenue : TMH,2025-07-17 02:28:35,"5,949,769,099","5,938,789,625",0,0,0,"10,979,474"


In [19]:
''' Mobile (Geo) '''

v_product_grp = 'Mobile'

mobile_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
mobile_geo_df = mobile_geo_df.loc[mobile_geo_df['METRIC_CD'].str.contains('GEO')]
mobile_geo_df = mobile_geo_df.loc[mobile_geo_df['TM_KEY_MTH']==v_tm_key_mth]
mobile_geo_df = mobile_geo_df.reset_index(drop=True)
mobile_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202506,Mobile,B0R000101GEO,Mobile Revenue (Geo),2025-07-17 05:19:06,0,"9,746,489,930","9,746,275,078","9,746,182,732","9,746,182,732","-9,746,489,930"
1,202506,Mobile,DB0R000101GEO,Mobile Revenue (Geo) : DTAC,2025-07-17 05:19:06,0,"4,208,435,900","4,208,433,303","4,208,364,673","4,208,364,673","-4,208,435,900"
2,202506,Mobile,TB0R000101GEO,Mobile Revenue (Geo) : TMH,2025-07-17 05:19:06,0,"5,538,054,030","5,537,841,776","5,537,818,059","5,537,818,059","-5,538,054,030"


### Prepaid

In [20]:
''' Prepaid '''

v_product_grp = 'Prepaid'

prepaid_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
prepaid_df = prepaid_df.loc[prepaid_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_df = prepaid_df.reset_index(drop=True)
prepaid_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202506,Prepaid,B1R000100,Prepaid Revenue,2025-07-17 05:19:06,"4,027,512,512","4,016,537,541","4,016,336,577","4,016,255,252","4,016,255,252","10,974,970"
1,202506,Prepaid,DB1R000100,Prepaid Revenue : DTAC,2025-07-17 05:19:06,"1,716,570,942","1,716,575,446","1,716,575,446","1,716,516,798","1,716,516,798","-4,503"
2,202506,Prepaid,TB1R000100,Prepaid Revenue : TMH,2025-07-17 05:19:06,"2,310,941,569","2,299,962,096","2,299,761,132","2,299,738,454","2,299,738,454","10,979,474"


### Postpaid

In [21]:
''' Postpaid '''

v_product_grp = 'Postpaid'

postpaid_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_df = postpaid_df.loc[~postpaid_df['METRIC_NAME'].str.contains('B2C|B2B')]
postpaid_df = postpaid_df.loc[postpaid_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_df = postpaid_df.reset_index(drop=True)
postpaid_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202506,Postpaid,B2R000100,Postpaid Revenue,2025-07-17 02:28:35,"6,324,928,718","6,324,928,718",0,0,0,0
1,202506,Postpaid,DB2R000100,Postpaid Revenue : DTAC,2025-07-17 05:19:06,"2,686,101,189","2,686,101,189","2,686,098,591","2,684,971,257","2,684,971,257",0
2,202506,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-07-17 02:28:35,"3,638,827,530","3,638,827,530",0,0,0,0


In [22]:
''' Postpaid B2C '''

v_product_grp = 'Postpaid'

postpaid_b2c_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['METRIC_NAME'].str.contains('B2C')]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2c_df = postpaid_b2c_df.reset_index(drop=True)
postpaid_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202506,Postpaid,B2R010100,Postpaid Revenue B2C,2025-07-17 05:19:06,"5,729,952,389","5,729,952,389","5,729,938,501","5,729,927,479","5,729,927,479",0
1,202506,Postpaid,DB2R010100,Postpaid Revenue B2C : DTAC,2025-07-17 05:19:06,"2,491,860,455","2,491,860,455","2,491,857,857","2,491,847,875","2,491,847,875",0
2,202506,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-07-17 05:19:06,"3,238,091,934","3,238,091,934","3,238,080,644","3,238,079,605","3,238,079,605",0


In [23]:
''' Postpaid B2B '''

v_product_grp = 'Postpaid'

postpaid_b2b_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['METRIC_NAME'].str.contains('B2B')]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2b_df = postpaid_b2b_df.reset_index(drop=True)
postpaid_b2b_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202506,Postpaid,B2R020100,Postpaid Revenue B2B,2025-07-17 02:28:35,"594,975,174","594,975,174",0,0,0,0
1,202506,Postpaid,DB2R020100,Postpaid Revenue B2B : DTAC,2025-07-17 05:19:06,"194,240,734","194,240,734","194,240,734","193,123,383","193,123,383",0
2,202506,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-07-17 02:28:35,"400,734,440","400,734,440",0,0,0,0


### TOL

In [24]:
''' TOL '''

v_product_grp = 'TOL'

tol_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_df = tol_df.loc[tol_df['TM_KEY_MTH']==v_tm_key_mth]
tol_df = tol_df.reset_index(drop=True)
tol_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202506,TOL,TB3R000100,TOL Revenue,2025-07-17 05:19:06,"1,610,862,222","1,606,978,956","1,606,978,956","1,606,978,956","1,606,978,956","3,883,267"


### TVS

In [25]:
''' TVS '''

v_product_grp = 'TVS'

tvs_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tvs_df = tvs_df.loc[tvs_df['TM_KEY_MTH']==v_tm_key_mth]
tvs_df = tvs_df.reset_index(drop=True)
tvs_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202506,TVS,TB4R000100,TVS Revenue,2025-07-17 05:19:06,"274,239,001","265,185,172","265,185,172","263,826,773","263,723,939","9,053,829"


## ** Special Metric (Corp & Geo)

### Revenue Corporate (C)
    B0R000100	Total Revenue
    DB0R000100	Total Revenue : DTAC
    TB0R000100	Total Revenue : TRUE
    
    B0R000101	Mobile Revenue
    DB0R000101	Mobile Revenue : DTAC
    TB0R000101	Mobile Revenue : TMH

In [26]:
''' B0R000100 : Total Revenue '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB1R000100', 'TB1R000100', 'DB2R000100', 'TB2R000100', 'TB3R000100', 'TB4R000100', 'TNSC00147')

v_metric_list = [
    'B0R000100' #Total Revenue
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'DB2R000100' #Postpaid Revenue : DTAC
    , 'TB2R000100' #Postpaid Revenue : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

total_rev_corp_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
total_rev_corp_df = total_rev_corp_df.loc[total_rev_corp_df['METRIC_CD'].isin(v_metric_list)]

total_rev_corp_df['TOTAL'] = np.where(total_rev_corp_df['METRIC_CD']=='B0R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['PRE_T'] = np.where(total_rev_corp_df['METRIC_CD']=='TB1R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['PRE_D'] = np.where(total_rev_corp_df['METRIC_CD']=='DB1R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['POST_T'] = np.where(total_rev_corp_df['METRIC_CD']=='TB2R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['POST_D'] = np.where(total_rev_corp_df['METRIC_CD']=='DB2R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['TOL'] = np.where(total_rev_corp_df['METRIC_CD']=='TB3R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['TVS'] = np.where(total_rev_corp_df['METRIC_CD']=='TB4R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['CHK_SUM'] = np.where(total_rev_corp_df['METRIC_CD'].isin(['TB1R000100','DB1R000100','TB2R000100','DB2R000100','TB3R000100','TB4R000100']), total_rev_corp_df['C'], 0)

total_rev_corp_df = total_rev_corp_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_T':'sum', 'POST_D':'sum', 'TOL':'sum', 'TVS':'sum', 'CHK_SUM':'sum'})
total_rev_corp_df['CHK_DIFF'] = total_rev_corp_df['TOTAL'] - total_rev_corp_df['CHK_SUM']
total_rev_corp_df = total_rev_corp_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_corp_df = total_rev_corp_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL', 'PRE_T', 'PRE_D', 'POST_T', 'POST_D', 'TOL', 'TVS']]

mod_col_list = total_rev_corp_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_corp_df[col] = total_rev_corp_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_corp_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL,PRE_T,PRE_D,POST_T,POST_D,TOL,TVS
0,202401,2025-07-17 05:19:06,0,"12,527,775,340","12,527,775,340","2,211,327,573","1,803,435,378","3,722,556,455","2,911,240,546","1,523,586,477","355,628,910"
1,202402,2025-07-17 05:19:06,0,"12,248,043,279","12,248,043,279","2,082,487,801","1,714,499,359","3,686,285,226","2,886,648,865","1,530,089,455","348,032,572"
2,202403,2025-07-17 05:19:06,0,"12,484,397,821","12,484,397,821","2,227,023,093","1,804,320,877","3,658,095,837","2,888,375,734","1,548,640,318","357,941,963"
3,202404,2025-07-17 05:19:06,0,"12,358,592,798","12,358,592,798","2,141,306,338","1,743,063,676","3,692,736,895","2,880,091,836","1,554,149,465","347,244,590"
4,202405,2025-07-17 05:19:06,0,"12,465,806,354","12,465,806,354","2,273,442,416","1,773,026,687","3,675,457,906","2,844,558,832","1,561,523,330","337,797,184"
5,202406,2025-07-17 05:19:06,0,"12,371,723,897","12,371,723,897","2,210,273,356","1,726,210,569","3,712,964,588","2,824,759,226","1,568,974,898","328,541,261"
6,202407,2025-07-17 05:19:06,0,"12,489,879,839","12,489,879,839","2,277,612,022","1,799,729,034","3,667,787,168","2,817,043,356","1,585,220,761","342,487,499"
7,202408,2025-07-17 05:19:06,0,"12,624,587,918","12,624,587,918","2,317,035,023","1,797,101,387","3,680,722,062","2,802,327,404","1,598,755,814","428,646,226"
8,202409,2025-07-17 05:19:06,0,"12,510,022,981","12,510,022,981","2,248,127,904","1,737,113,658","3,706,292,256","2,784,939,778","1,641,741,349","391,808,037"
9,202410,2025-07-17 05:19:06,0,"12,569,510,549","12,569,510,549","2,349,377,919","1,782,549,687","3,685,237,458","2,800,494,445","1,614,615,686","337,235,354"


In [27]:
''' TB0R000100 : Total Revenue : TRUE '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('TB1R000100', 'TB2R000100', 'TB3R000100', 'TB4R000100')

v_metric_list = [
    'TB0R000100' #Total Revenue : TRUE
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'TB2R000100' #Postpaid Revenue : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

total_rev_corp_true_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
total_rev_corp_true_df = total_rev_corp_true_df.loc[total_rev_corp_true_df['METRIC_CD'].isin(v_metric_list)]

total_rev_corp_true_df['TOTAL_T'] = np.where(total_rev_corp_true_df['METRIC_CD']=='TB0R000100', total_rev_corp_true_df['C'], 0)
total_rev_corp_true_df['PRE_T'] = np.where(total_rev_corp_true_df['METRIC_CD']=='TB1R000100', total_rev_corp_true_df['C'], 0)
total_rev_corp_true_df['POST_T'] = np.where(total_rev_corp_true_df['METRIC_CD']=='TB2R000100', total_rev_corp_true_df['C'], 0)
total_rev_corp_true_df['TOL'] = np.where(total_rev_corp_true_df['METRIC_CD']=='TB3R000100', total_rev_corp_true_df['C'], 0)
total_rev_corp_true_df['TVS'] = np.where(total_rev_corp_true_df['METRIC_CD']=='TB4R000100', total_rev_corp_true_df['C'], 0)
total_rev_corp_true_df['CHK_SUM'] = np.where(total_rev_corp_true_df['METRIC_CD'].isin(['TB1R000100','TB2R000100','TB3R000100','TB4R000100']), total_rev_corp_true_df['C'], 0)

total_rev_corp_true_df = total_rev_corp_true_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_T':'sum', 'PRE_T':'sum', 'POST_T':'sum', 'TOL':'sum', 'TVS':'sum', 'CHK_SUM':'sum'})
total_rev_corp_true_df['CHK_DIFF'] = total_rev_corp_true_df['TOTAL_T'] - total_rev_corp_true_df['CHK_SUM']
total_rev_corp_true_df = total_rev_corp_true_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_corp_true_df = total_rev_corp_true_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_T', 'PRE_T', 'POST_T', 'TOL', 'TVS']]

mod_col_list = total_rev_corp_true_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_corp_true_df[col] = total_rev_corp_true_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_corp_true_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_T,PRE_T,POST_T,TOL,TVS
0,202401,2025-07-17 05:19:06,-0,"7,813,099,416","7,813,099,416","2,211,327,573","3,722,556,455","1,523,586,477","355,628,910"
1,202402,2025-07-17 05:19:06,-0,"7,646,895,055","7,646,895,055","2,082,487,801","3,686,285,226","1,530,089,455","348,032,572"
2,202403,2025-07-17 05:19:06,0,"7,791,701,211","7,791,701,211","2,227,023,093","3,658,095,837","1,548,640,318","357,941,963"
3,202404,2025-07-17 05:19:06,0,"7,735,437,287","7,735,437,287","2,141,306,338","3,692,736,895","1,554,149,465","347,244,590"
4,202405,2025-07-17 05:19:06,0,"7,848,220,835","7,848,220,835","2,273,442,416","3,675,457,906","1,561,523,330","337,797,184"
5,202406,2025-07-17 05:19:06,0,"7,820,754,103","7,820,754,103","2,210,273,356","3,712,964,588","1,568,974,898","328,541,261"
6,202407,2025-07-17 05:19:06,0,"7,873,107,450","7,873,107,450","2,277,612,022","3,667,787,168","1,585,220,761","342,487,499"
7,202408,2025-07-17 05:19:06,0,"8,025,159,126","8,025,159,126","2,317,035,023","3,680,722,062","1,598,755,814","428,646,226"
8,202409,2025-07-17 05:19:06,0,"7,987,969,545","7,987,969,545","2,248,127,904","3,706,292,256","1,641,741,349","391,808,037"
9,202410,2025-07-17 05:19:06,0,"7,986,466,416","7,986,466,416","2,349,377,919","3,685,237,458","1,614,615,686","337,235,354"


In [28]:
''' DB0R000100 : Total Revenue : DTAC '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB1R000100', 'DB2R000100')

v_metric_list = [
    'DB0R000100' #Total Revenue : DTAC
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'DB2R000100' #Postpaid Revenue : DTAC
    ]

total_rev_corp_dtac_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
total_rev_corp_dtac_df = total_rev_corp_dtac_df.loc[total_rev_corp_dtac_df['METRIC_CD'].isin(v_metric_list)]

total_rev_corp_dtac_df['TOTAL_D'] = np.where(total_rev_corp_dtac_df['METRIC_CD']=='DB0R000100', total_rev_corp_dtac_df['C'], 0)
total_rev_corp_dtac_df['PRE_D'] = np.where(total_rev_corp_dtac_df['METRIC_CD']=='DB1R000100', total_rev_corp_dtac_df['C'], 0)
total_rev_corp_dtac_df['POST_D'] = np.where(total_rev_corp_dtac_df['METRIC_CD']=='DB2R000100', total_rev_corp_dtac_df['C'], 0)
total_rev_corp_dtac_df['CHK_SUM'] = np.where(total_rev_corp_dtac_df['METRIC_CD'].isin(['DB1R000100','DB2R000100']), total_rev_corp_dtac_df['C'], 0)

total_rev_corp_dtac_df = total_rev_corp_dtac_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_D':'sum', 'PRE_D':'sum', 'POST_D':'sum', 'CHK_SUM':'sum'})
total_rev_corp_dtac_df['CHK_DIFF'] = total_rev_corp_dtac_df['TOTAL_D'] - total_rev_corp_dtac_df['CHK_SUM']
total_rev_corp_dtac_df = total_rev_corp_dtac_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_corp_dtac_df = total_rev_corp_dtac_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_D', 'PRE_D', 'POST_D']]

mod_col_list = total_rev_corp_dtac_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_corp_dtac_df[col] = total_rev_corp_dtac_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_corp_dtac_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_D,PRE_D,POST_D
0,202401,2025-07-17 05:19:06,0,"4,714,675,924","4,714,675,924","1,803,435,378","2,911,240,546"
1,202402,2025-07-17 05:19:06,0,"4,601,148,224","4,601,148,224","1,714,499,359","2,886,648,865"
2,202403,2025-07-17 05:19:06,0,"4,692,696,611","4,692,696,611","1,804,320,877","2,888,375,734"
3,202404,2025-07-17 05:19:06,0,"4,623,155,511","4,623,155,511","1,743,063,676","2,880,091,836"
4,202405,2025-07-17 05:19:06,0,"4,617,585,519","4,617,585,519","1,773,026,687","2,844,558,832"
5,202406,2025-07-17 05:19:06,0,"4,550,969,794","4,550,969,794","1,726,210,569","2,824,759,226"
6,202407,2025-07-17 05:19:06,0,"4,616,772,389","4,616,772,389","1,799,729,034","2,817,043,356"
7,202408,2025-07-17 05:19:06,0,"4,599,428,792","4,599,428,792","1,797,101,387","2,802,327,404"
8,202409,2025-07-17 05:19:06,0,"4,522,053,436","4,522,053,436","1,737,113,658","2,784,939,778"
9,202410,2025-07-17 05:19:06,0,"4,583,044,133","4,583,044,133","1,782,549,687","2,800,494,445"


In [29]:
''' B0R000101 : Mobile Revenue '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB2R000100', 'TB2R000100', 'DB1R000100', 'TB1R000100')

v_metric_list = [
    'B0R000101' #Mobile Revenue
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'DB2R000100' #Postpaid Revenue : DTAC
    , 'TB2R000100' #Postpaid Revenue : TMH
    ]

mobile_rev_corp_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
mobile_rev_corp_df = mobile_rev_corp_df.loc[mobile_rev_corp_df['METRIC_CD'].isin(v_metric_list)]

mobile_rev_corp_df['MOBILE'] = np.where(mobile_rev_corp_df['METRIC_CD']=='B0R000101', mobile_rev_corp_df['C'], 0)
mobile_rev_corp_df['PRE_T'] = np.where(mobile_rev_corp_df['METRIC_CD']=='TB1R000100', mobile_rev_corp_df['C'], 0)
mobile_rev_corp_df['PRE_D'] = np.where(mobile_rev_corp_df['METRIC_CD']=='DB1R000100', mobile_rev_corp_df['C'], 0)
mobile_rev_corp_df['POST_T'] = np.where(mobile_rev_corp_df['METRIC_CD']=='TB2R000100', mobile_rev_corp_df['C'], 0)
mobile_rev_corp_df['POST_D'] = np.where(mobile_rev_corp_df['METRIC_CD']=='DB2R000100', mobile_rev_corp_df['C'], 0)
mobile_rev_corp_df['CHK_SUM'] = np.where(mobile_rev_corp_df['METRIC_CD'].isin(['TB1R000100','DB1R000100','TB2R000100','DB2R000100']), mobile_rev_corp_df['C'], 0)

mobile_rev_corp_df = mobile_rev_corp_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_T':'sum', 'POST_D':'sum', 'CHK_SUM':'sum'})
mobile_rev_corp_df['CHK_DIFF'] = mobile_rev_corp_df['MOBILE'] - mobile_rev_corp_df['CHK_SUM']
mobile_rev_corp_df = mobile_rev_corp_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_corp_df = mobile_rev_corp_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE', 'PRE_T', 'PRE_D', 'POST_T', 'POST_D']]

mod_col_list = mobile_rev_corp_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_corp_df[col] = mobile_rev_corp_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_corp_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE,PRE_T,PRE_D,POST_T,POST_D
0,202401,2025-07-17 05:19:06,0,"10,648,559,952","10,648,559,952","2,211,327,573","1,803,435,378","3,722,556,455","2,911,240,546"
1,202402,2025-07-17 05:19:06,0,"10,369,921,251","10,369,921,251","2,082,487,801","1,714,499,359","3,686,285,226","2,886,648,865"
2,202403,2025-07-17 05:19:06,0,"10,577,815,541","10,577,815,541","2,227,023,093","1,804,320,877","3,658,095,837","2,888,375,734"
3,202404,2025-07-17 05:19:06,0,"10,457,198,744","10,457,198,744","2,141,306,338","1,743,063,676","3,692,736,895","2,880,091,836"
4,202405,2025-07-17 05:19:06,0,"10,566,485,841","10,566,485,841","2,273,442,416","1,773,026,687","3,675,457,906","2,844,558,832"
5,202406,2025-07-17 05:19:06,0,"10,474,207,738","10,474,207,738","2,210,273,356","1,726,210,569","3,712,964,588","2,824,759,226"
6,202407,2025-07-17 05:19:06,0,"10,562,171,579","10,562,171,579","2,277,612,022","1,799,729,034","3,667,787,168","2,817,043,356"
7,202408,2025-07-17 05:19:06,0,"10,597,185,877","10,597,185,877","2,317,035,023","1,797,101,387","3,680,722,062","2,802,327,404"
8,202409,2025-07-17 05:19:06,0,"10,476,473,596","10,476,473,596","2,248,127,904","1,737,113,658","3,706,292,256","2,784,939,778"
9,202410,2025-07-17 05:19:06,0,"10,617,659,509","10,617,659,509","2,349,377,919","1,782,549,687","3,685,237,458","2,800,494,445"


In [30]:
''' TB0R000101 : Mobile Revenue : TMH '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('TB2R000100', 'TB1R000100')

v_metric_list = [
    'TB0R000101' #Mobile Revenue : TMH
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'TB2R000100' #Postpaid Revenue : TMH
    ]

mobile_rev_corp_tmh_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
mobile_rev_corp_tmh_df = mobile_rev_corp_tmh_df.loc[mobile_rev_corp_tmh_df['METRIC_CD'].isin(v_metric_list)]

mobile_rev_corp_tmh_df['MOBILE_T'] = np.where(mobile_rev_corp_tmh_df['METRIC_CD']=='TB0R000101', mobile_rev_corp_tmh_df['C'], 0)
mobile_rev_corp_tmh_df['PRE_T'] = np.where(mobile_rev_corp_tmh_df['METRIC_CD']=='TB1R000100', mobile_rev_corp_tmh_df['C'], 0)
mobile_rev_corp_tmh_df['POST_T'] = np.where(mobile_rev_corp_tmh_df['METRIC_CD']=='TB2R000100', mobile_rev_corp_tmh_df['C'], 0)
mobile_rev_corp_tmh_df['CHK_SUM'] = np.where(mobile_rev_corp_tmh_df['METRIC_CD'].isin(['TB1R000100','TB2R000100']), mobile_rev_corp_tmh_df['C'], 0)

mobile_rev_corp_tmh_df = mobile_rev_corp_tmh_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE_T':'sum', 'PRE_T':'sum', 'POST_T':'sum', 'CHK_SUM':'sum'})
mobile_rev_corp_tmh_df['CHK_DIFF'] = mobile_rev_corp_tmh_df['MOBILE_T'] - mobile_rev_corp_tmh_df['CHK_SUM']
mobile_rev_corp_tmh_df = mobile_rev_corp_tmh_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_corp_tmh_df = mobile_rev_corp_tmh_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE_T', 'PRE_T', 'POST_T']]

mod_col_list = mobile_rev_corp_tmh_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_corp_tmh_df[col] = mobile_rev_corp_tmh_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_corp_tmh_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE_T,PRE_T,POST_T
0,202401,2025-07-17 05:19:06,0,"5,933,884,028","5,933,884,028","2,211,327,573","3,722,556,455"
1,202402,2025-07-17 05:19:06,0,"5,768,773,028","5,768,773,028","2,082,487,801","3,686,285,226"
2,202403,2025-07-17 05:19:06,0,"5,885,118,930","5,885,118,930","2,227,023,093","3,658,095,837"
3,202404,2025-07-17 05:19:06,0,"5,834,043,232","5,834,043,232","2,141,306,338","3,692,736,895"
4,202405,2025-07-17 05:19:06,0,"5,948,900,322","5,948,900,322","2,273,442,416","3,675,457,906"
5,202406,2025-07-17 05:19:06,0,"5,923,237,944","5,923,237,944","2,210,273,356","3,712,964,588"
6,202407,2025-07-17 05:19:06,-0,"5,945,399,190","5,945,399,190","2,277,612,022","3,667,787,168"
7,202408,2025-07-17 05:19:06,0,"5,997,757,085","5,997,757,085","2,317,035,023","3,680,722,062"
8,202409,2025-07-17 05:19:06,0,"5,954,420,160","5,954,420,160","2,248,127,904","3,706,292,256"
9,202410,2025-07-17 05:19:06,0,"6,034,615,376","6,034,615,376","2,349,377,919","3,685,237,458"


In [31]:
''' DB0R000101 : Mobile Revenue : DTAC '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB2R000100', 'DB1R000100')

v_metric_list = [
    'DB0R000101' #Mobile Revenue : DTAC
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'DB2R000100' #Postpaid Revenue : DTAC
    ]

mobile_rev_corp_dtac_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
mobile_rev_corp_dtac_df = mobile_rev_corp_dtac_df.loc[mobile_rev_corp_dtac_df['METRIC_CD'].isin(v_metric_list)]

mobile_rev_corp_dtac_df['MOBILE_D'] = np.where(mobile_rev_corp_dtac_df['METRIC_CD']=='DB0R000101', mobile_rev_corp_dtac_df['C'], 0)
mobile_rev_corp_dtac_df['PRE_D'] = np.where(mobile_rev_corp_dtac_df['METRIC_CD']=='DB1R000100', mobile_rev_corp_dtac_df['C'], 0)
mobile_rev_corp_dtac_df['POST_D'] = np.where(mobile_rev_corp_dtac_df['METRIC_CD']=='DB2R000100', mobile_rev_corp_dtac_df['C'], 0)
mobile_rev_corp_dtac_df['CHK_SUM'] = np.where(mobile_rev_corp_dtac_df['METRIC_CD'].isin(['DB1R000100','DB2R000100']), mobile_rev_corp_dtac_df['C'], 0)

mobile_rev_corp_dtac_df = mobile_rev_corp_dtac_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE_D':'sum', 'PRE_D':'sum', 'POST_D':'sum', 'CHK_SUM':'sum'})
mobile_rev_corp_dtac_df['CHK_DIFF'] = mobile_rev_corp_dtac_df['MOBILE_D'] - mobile_rev_corp_dtac_df['CHK_SUM']
mobile_rev_corp_dtac_df = mobile_rev_corp_dtac_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_corp_dtac_df = mobile_rev_corp_dtac_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE_D', 'PRE_D', 'POST_D']]

mod_col_list = mobile_rev_corp_dtac_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_corp_dtac_df[col] = mobile_rev_corp_dtac_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_corp_dtac_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE_D,PRE_D,POST_D
0,202401,2025-07-17 05:19:06,0,"4,714,675,924","4,714,675,924","1,803,435,378","2,911,240,546"
1,202402,2025-07-17 05:19:06,0,"4,601,148,224","4,601,148,224","1,714,499,359","2,886,648,865"
2,202403,2025-07-17 05:19:06,0,"4,692,696,611","4,692,696,611","1,804,320,877","2,888,375,734"
3,202404,2025-07-17 05:19:06,0,"4,623,155,511","4,623,155,511","1,743,063,676","2,880,091,836"
4,202405,2025-07-17 05:19:06,0,"4,617,585,519","4,617,585,519","1,773,026,687","2,844,558,832"
5,202406,2025-07-17 05:19:06,0,"4,550,969,794","4,550,969,794","1,726,210,569","2,824,759,226"
6,202407,2025-07-17 05:19:06,0,"4,616,772,389","4,616,772,389","1,799,729,034","2,817,043,356"
7,202408,2025-07-17 05:19:06,0,"4,599,428,792","4,599,428,792","1,797,101,387","2,802,327,404"
8,202409,2025-07-17 05:19:06,0,"4,522,053,436","4,522,053,436","1,737,113,658","2,784,939,778"
9,202410,2025-07-17 05:19:06,0,"4,583,044,133","4,583,044,133","1,782,549,687","2,800,494,445"


### Revenue Geo (P)
    B0R000100GEO	Total Revenue (Geo)
    DB0R000100GEO	Total Revenue (Geo) : DTAC
    TB0R000100GEO	Total Revenue (Geo) : TRUE
    
    B0R000101GEO	Mobile Revenue (Geo)
    DB0R000101GEO	Mobile Revenue (Geo) : DTAC
    TB0R000101GEO	Mobile Revenue (Geo) : TMH

In [32]:
''' B0R000100GEO : Total Revenue (Geo) '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

total_rev_geo_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_rev_geo_df = total_rev_geo_df.loc[total_rev_geo_df['METRIC_CD'].isin(v_metric_list)]

total_rev_geo_df['TOTAL'] = np.where(total_rev_geo_df['METRIC_CD']=='B0R000100GEO', total_rev_geo_df['P'], 0)
total_rev_geo_df['PRE_T'] = np.where(total_rev_geo_df['METRIC_CD']=='TB1R000100', total_rev_geo_df['P'], 0)
total_rev_geo_df['PRE_D'] = np.where(total_rev_geo_df['METRIC_CD']=='DB1R000100', total_rev_geo_df['P'], 0)
total_rev_geo_df['POST_B2C_T'] = np.where(total_rev_geo_df['METRIC_CD']=='TB2R010100', total_rev_geo_df['P'], 0)
total_rev_geo_df['POST_B2C_D'] = np.where(total_rev_geo_df['METRIC_CD']=='DB2R010100', total_rev_geo_df['P'], 0)
total_rev_geo_df['TOL'] = np.where(total_rev_geo_df['METRIC_CD']=='TB3R000100', total_rev_geo_df['P'], 0)
total_rev_geo_df['TVS'] = np.where(total_rev_geo_df['METRIC_CD']=='TB4R000100', total_rev_geo_df['P'], 0)
total_rev_geo_df['CHK_SUM'] = np.where(total_rev_geo_df['METRIC_CD'].isin(['TB1R000100','DB1R000100','TB2R010100','DB2R010100','TB3R000100','TB4R000100']), total_rev_geo_df['P'], 0)

total_rev_geo_df = total_rev_geo_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL':'sum', 'TVS':'sum', 'CHK_SUM':'sum'})
total_rev_geo_df['CHK_DIFF'] = total_rev_geo_df['TOTAL'] - total_rev_geo_df['CHK_SUM']
total_rev_geo_df = total_rev_geo_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_geo_df = total_rev_geo_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL', 'PRE_T', 'PRE_D', 'POST_B2C_T', 'POST_B2C_D', 'TOL', 'TVS']]

mod_col_list = total_rev_geo_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_geo_df[col] = total_rev_geo_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_geo_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL,PRE_T,PRE_D,POST_B2C_T,POST_B2C_D,TOL,TVS
0,202401,2025-07-17 05:19:06,0,"11,881,730,399","11,881,730,399","2,211,126,641","1,803,547,883","3,311,282,032","2,709,538,268","1,507,069,397","339,166,179"
1,202402,2025-07-17 05:19:06,0,"11,626,563,733","11,626,563,733","2,082,104,053","1,714,609,789","3,285,072,468","2,689,888,663","1,523,039,176","331,849,584"
2,202403,2025-07-17 05:19:06,0,"11,853,639,343","11,853,639,343","2,226,380,294","1,804,408,125","3,254,097,222","2,686,745,365","1,539,811,431","342,196,905"
3,202404,2025-07-17 05:19:06,0,"11,723,479,404","11,723,479,404","2,140,030,223","1,743,154,862","3,280,178,978","2,681,407,303","1,546,917,278","331,790,759"
4,202405,2025-07-17 05:19:06,0,"11,841,867,846","11,841,867,846","2,269,268,086","1,773,106,846","3,273,314,626","2,648,304,046","1,554,971,583","322,902,658"
5,202406,2025-07-17 05:19:06,0,"11,719,185,282","11,719,185,282","2,206,174,307","1,726,295,968","3,284,957,272","2,623,975,210","1,563,892,736","313,889,791"
6,202407,2025-07-17 05:19:06,0,"11,857,212,955","11,857,212,955","2,272,833,725","1,799,831,136","3,269,232,707","2,609,495,087","1,577,812,706","328,007,594"
7,202408,2025-07-17 05:19:06,-0,"11,992,892,504","11,992,892,504","2,312,752,271","1,797,206,658","3,283,314,789","2,597,627,729","1,592,482,287","409,508,769"
8,202409,2025-07-17 05:19:06,0,"11,856,710,578","11,856,710,578","2,245,163,825","1,737,197,388","3,281,480,710","2,586,882,098","1,633,490,490","372,496,067"
9,202410,2025-07-17 05:19:06,0,"11,900,384,524","11,900,384,524","2,345,529,988","1,782,644,560","3,272,186,553","2,590,323,955","1,598,256,224","311,443,243"


In [33]:
''' TB0R000100GEO : Total Revenue (Geo) : TRUE '''

v_metric_list = [
    'TB0R000100GEO' #Total Revenue (Geo) : TRUE
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

total_rev_geo_true_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_rev_geo_true_df = total_rev_geo_true_df.loc[total_rev_geo_true_df['METRIC_CD'].isin(v_metric_list)]

total_rev_geo_true_df['TOTAL_T'] = np.where(total_rev_geo_true_df['METRIC_CD']=='TB0R000100GEO', total_rev_geo_true_df['P'], 0)
total_rev_geo_true_df['PRE_T'] = np.where(total_rev_geo_true_df['METRIC_CD']=='TB1R000100', total_rev_geo_true_df['P'], 0)
total_rev_geo_true_df['POST_B2C_T'] = np.where(total_rev_geo_true_df['METRIC_CD']=='TB2R010100', total_rev_geo_true_df['P'], 0)
total_rev_geo_true_df['TOL'] = np.where(total_rev_geo_true_df['METRIC_CD']=='TB3R000100', total_rev_geo_true_df['P'], 0)
total_rev_geo_true_df['TVS'] = np.where(total_rev_geo_true_df['METRIC_CD']=='TB4R000100', total_rev_geo_true_df['P'], 0)
total_rev_geo_true_df['CHK_SUM'] = np.where(total_rev_geo_true_df['METRIC_CD'].isin(['TB1R000100','TB2R010100','TB3R000100','TB4R000100']), total_rev_geo_true_df['P'], 0)

total_rev_geo_true_df = total_rev_geo_true_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_T':'sum', 'PRE_T':'sum', 'POST_B2C_T':'sum', 'TOL':'sum', 'TVS':'sum', 'CHK_SUM':'sum'})
total_rev_geo_true_df['CHK_DIFF'] = total_rev_geo_true_df['TOTAL_T'] - total_rev_geo_true_df['CHK_SUM']
total_rev_geo_true_df = total_rev_geo_true_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_geo_true_df = total_rev_geo_true_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_T', 'PRE_T', 'POST_B2C_T', 'TOL', 'TVS']]

mod_col_list = total_rev_geo_true_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_geo_true_df[col] = total_rev_geo_true_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_geo_true_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_T,PRE_T,POST_B2C_T,TOL,TVS
0,202401,2025-07-17 05:19:06,0,"7,368,644,248","7,368,644,248","2,211,126,641","3,311,282,032","1,507,069,397","339,166,179"
1,202402,2025-07-17 05:19:06,0,"7,222,065,281","7,222,065,281","2,082,104,053","3,285,072,468","1,523,039,176","331,849,584"
2,202403,2025-07-17 05:19:06,0,"7,362,485,852","7,362,485,852","2,226,380,294","3,254,097,222","1,539,811,431","342,196,905"
3,202404,2025-07-17 05:19:06,0,"7,298,917,238","7,298,917,238","2,140,030,223","3,280,178,978","1,546,917,278","331,790,759"
4,202405,2025-07-17 05:19:06,0,"7,420,456,954","7,420,456,954","2,269,268,086","3,273,314,626","1,554,971,583","322,902,658"
5,202406,2025-07-17 05:19:06,0,"7,368,914,105","7,368,914,105","2,206,174,307","3,284,957,272","1,563,892,736","313,889,791"
6,202407,2025-07-17 05:19:06,0,"7,447,886,733","7,447,886,733","2,272,833,725","3,269,232,707","1,577,812,706","328,007,594"
7,202408,2025-07-17 05:19:06,0,"7,598,058,116","7,598,058,116","2,312,752,271","3,283,314,789","1,592,482,287","409,508,769"
8,202409,2025-07-17 05:19:06,0,"7,532,631,092","7,532,631,092","2,245,163,825","3,281,480,710","1,633,490,490","372,496,067"
9,202410,2025-07-17 05:19:06,-0,"7,527,416,008","7,527,416,008","2,345,529,988","3,272,186,553","1,598,256,224","311,443,243"


In [34]:
''' DB0R000100GEO : Total Revenue (Geo) : DTAC '''

v_metric_list = [
    'DB0R000100GEO' #Total Revenue (Geo) : DTAC
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    ]

total_rev_geo_dtac_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_rev_geo_dtac_df = total_rev_geo_dtac_df.loc[total_rev_geo_dtac_df['METRIC_CD'].isin(v_metric_list)]

total_rev_geo_dtac_df['TOTAL_D'] = np.where(total_rev_geo_dtac_df['METRIC_CD']=='DB0R000100GEO', total_rev_geo_dtac_df['P'], 0)
total_rev_geo_dtac_df['PRE_D'] = np.where(total_rev_geo_dtac_df['METRIC_CD']=='DB1R000100', total_rev_geo_dtac_df['P'], 0)
total_rev_geo_dtac_df['POST_B2C_D'] = np.where(total_rev_geo_dtac_df['METRIC_CD']=='DB2R010100', total_rev_geo_dtac_df['P'], 0)
total_rev_geo_dtac_df['CHK_SUM'] = np.where(total_rev_geo_dtac_df['METRIC_CD'].isin(['DB1R000100','DB2R010100']), total_rev_geo_dtac_df['P'], 0)

total_rev_geo_dtac_df = total_rev_geo_dtac_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_D':'sum', 'PRE_D':'sum', 'POST_B2C_D':'sum', 'CHK_SUM':'sum'})
total_rev_geo_dtac_df['CHK_DIFF'] = total_rev_geo_dtac_df['TOTAL_D'] - total_rev_geo_dtac_df['CHK_SUM']
total_rev_geo_dtac_df = total_rev_geo_dtac_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_geo_dtac_df = total_rev_geo_dtac_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_D', 'PRE_D', 'POST_B2C_D']]

mod_col_list = total_rev_geo_dtac_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_geo_dtac_df[col] = total_rev_geo_dtac_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_geo_dtac_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_D,PRE_D,POST_B2C_D
0,202401,2025-07-17 05:19:06,0,"4,513,086,151","4,513,086,151","1,803,547,883","2,709,538,268"
1,202402,2025-07-17 05:19:06,0,"4,404,498,452","4,404,498,452","1,714,609,789","2,689,888,663"
2,202403,2025-07-17 05:19:06,0,"4,491,153,491","4,491,153,491","1,804,408,125","2,686,745,365"
3,202404,2025-07-17 05:19:06,0,"4,424,562,165","4,424,562,165","1,743,154,862","2,681,407,303"
4,202405,2025-07-17 05:19:06,0,"4,421,410,892","4,421,410,892","1,773,106,846","2,648,304,046"
5,202406,2025-07-17 05:19:06,0,"4,350,271,177","4,350,271,177","1,726,295,968","2,623,975,210"
6,202407,2025-07-17 05:19:06,0,"4,409,326,223","4,409,326,223","1,799,831,136","2,609,495,087"
7,202408,2025-07-17 05:19:06,0,"4,394,834,388","4,394,834,388","1,797,206,658","2,597,627,729"
8,202409,2025-07-17 05:19:06,-0,"4,324,079,486","4,324,079,486","1,737,197,388","2,586,882,098"
9,202410,2025-07-17 05:19:06,0,"4,372,968,515","4,372,968,515","1,782,644,560","2,590,323,955"


In [35]:
''' B0R000101GEO : Mobile Revenue (Geo) '''

v_metric_list = [
    'B0R000101GEO' #Mobile Revenue (Geo)
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    ]

mobile_rev_geo_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
mobile_rev_geo_df = mobile_rev_geo_df.loc[mobile_rev_geo_df['METRIC_CD'].isin(v_metric_list)]

mobile_rev_geo_df['MOBILE'] = np.where(mobile_rev_geo_df['METRIC_CD']=='B0R000101GEO', mobile_rev_geo_df['P'], 0)
mobile_rev_geo_df['PRE_T'] = np.where(mobile_rev_geo_df['METRIC_CD']=='TB1R000100', mobile_rev_geo_df['P'], 0)
mobile_rev_geo_df['PRE_D'] = np.where(mobile_rev_geo_df['METRIC_CD']=='DB1R000100', mobile_rev_geo_df['P'], 0)
mobile_rev_geo_df['POST_B2C_T'] = np.where(mobile_rev_geo_df['METRIC_CD']=='TB2R010100', mobile_rev_geo_df['P'], 0)
mobile_rev_geo_df['POST_B2C_D'] = np.where(mobile_rev_geo_df['METRIC_CD']=='DB2R010100', mobile_rev_geo_df['P'], 0)
mobile_rev_geo_df['CHK_SUM'] = np.where(mobile_rev_geo_df['METRIC_CD'].isin(['TB1R000100','DB1R000100','TB2R010100','DB2R010100']), mobile_rev_geo_df['P'], 0)

mobile_rev_geo_df = mobile_rev_geo_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'CHK_SUM':'sum'})
mobile_rev_geo_df['CHK_DIFF'] = mobile_rev_geo_df['MOBILE'] - mobile_rev_geo_df['CHK_SUM']
mobile_rev_geo_df = mobile_rev_geo_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_geo_df = mobile_rev_geo_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE', 'PRE_T', 'PRE_D', 'POST_B2C_T', 'POST_B2C_D']]

mod_col_list = mobile_rev_geo_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_geo_df[col] = mobile_rev_geo_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_geo_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE,PRE_T,PRE_D,POST_B2C_T,POST_B2C_D
0,202401,2025-07-17 05:19:06,0,"10,035,494,824","10,035,494,824","2,211,126,641","1,803,547,883","3,311,282,032","2,709,538,268"
1,202402,2025-07-17 05:19:06,0,"9,771,674,973","9,771,674,973","2,082,104,053","1,714,609,789","3,285,072,468","2,689,888,663"
2,202403,2025-07-17 05:19:06,0,"9,971,631,007","9,971,631,007","2,226,380,294","1,804,408,125","3,254,097,222","2,686,745,365"
3,202404,2025-07-17 05:19:06,0,"9,844,771,366","9,844,771,366","2,140,030,223","1,743,154,862","3,280,178,978","2,681,407,303"
4,202405,2025-07-17 05:19:06,-0,"9,963,993,605","9,963,993,605","2,269,268,086","1,773,106,846","3,273,314,626","2,648,304,046"
5,202406,2025-07-17 05:19:06,0,"9,841,402,756","9,841,402,756","2,206,174,307","1,726,295,968","3,284,957,272","2,623,975,210"
6,202407,2025-07-17 05:19:06,0,"9,951,392,655","9,951,392,655","2,272,833,725","1,799,831,136","3,269,232,707","2,609,495,087"
7,202408,2025-07-17 05:19:06,0,"9,990,901,448","9,990,901,448","2,312,752,271","1,797,206,658","3,283,314,789","2,597,627,729"
8,202409,2025-07-17 05:19:06,0,"9,850,724,021","9,850,724,021","2,245,163,825","1,737,197,388","3,281,480,710","2,586,882,098"
9,202410,2025-07-17 05:19:06,0,"9,990,685,057","9,990,685,057","2,345,529,988","1,782,644,560","3,272,186,553","2,590,323,955"


In [36]:
''' TB0R000101GEO : Mobile Revenue (Geo) : TMH '''

v_metric_list = [
    'TB0R000101GEO' #Mobile Revenue (Geo) : TMH
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    ]

mobile_rev_geo_tmh_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
mobile_rev_geo_tmh_df = mobile_rev_geo_tmh_df.loc[mobile_rev_geo_tmh_df['METRIC_CD'].isin(v_metric_list)]

mobile_rev_geo_tmh_df['MOBILE_T'] = np.where(mobile_rev_geo_tmh_df['METRIC_CD']=='TB0R000101GEO', mobile_rev_geo_tmh_df['P'], 0)
mobile_rev_geo_tmh_df['PRE_T'] = np.where(mobile_rev_geo_tmh_df['METRIC_CD']=='TB1R000100', mobile_rev_geo_tmh_df['P'], 0)
mobile_rev_geo_tmh_df['POST_B2C_T'] = np.where(mobile_rev_geo_tmh_df['METRIC_CD']=='TB2R010100', mobile_rev_geo_tmh_df['P'], 0)
mobile_rev_geo_tmh_df['CHK_SUM'] = np.where(mobile_rev_geo_tmh_df['METRIC_CD'].isin(['TB1R000100','TB2R010100']), mobile_rev_geo_tmh_df['P'], 0)

mobile_rev_geo_tmh_df = mobile_rev_geo_tmh_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE_T':'sum', 'PRE_T':'sum', 'POST_B2C_T':'sum', 'CHK_SUM':'sum'})
mobile_rev_geo_tmh_df['CHK_DIFF'] = mobile_rev_geo_tmh_df['MOBILE_T'] - mobile_rev_geo_tmh_df['CHK_SUM']
mobile_rev_geo_tmh_df = mobile_rev_geo_tmh_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_geo_tmh_df = mobile_rev_geo_tmh_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE_T', 'PRE_T', 'POST_B2C_T']]

mod_col_list = mobile_rev_geo_tmh_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_geo_tmh_df[col] = mobile_rev_geo_tmh_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_geo_tmh_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE_T,PRE_T,POST_B2C_T
0,202401,2025-07-17 05:19:06,0,"5,522,408,673","5,522,408,673","2,211,126,641","3,311,282,032"
1,202402,2025-07-17 05:19:06,-0,"5,367,176,521","5,367,176,521","2,082,104,053","3,285,072,468"
2,202403,2025-07-17 05:19:06,0,"5,480,477,516","5,480,477,516","2,226,380,294","3,254,097,222"
3,202404,2025-07-17 05:19:06,0,"5,420,209,201","5,420,209,201","2,140,030,223","3,280,178,978"
4,202405,2025-07-17 05:19:06,0,"5,542,582,712","5,542,582,712","2,269,268,086","3,273,314,626"
5,202406,2025-07-17 05:19:06,0,"5,491,131,578","5,491,131,578","2,206,174,307","3,284,957,272"
6,202407,2025-07-17 05:19:06,0,"5,542,066,432","5,542,066,432","2,272,833,725","3,269,232,707"
7,202408,2025-07-17 05:19:06,0,"5,596,067,060","5,596,067,060","2,312,752,271","3,283,314,789"
8,202409,2025-07-17 05:19:06,0,"5,526,644,535","5,526,644,535","2,245,163,825","3,281,480,710"
9,202410,2025-07-17 05:19:06,0,"5,617,716,541","5,617,716,541","2,345,529,988","3,272,186,553"


In [37]:
''' DB0R000101GEO : Mobile Revenue (Geo) : DTAC '''

v_metric_list = [
    'DB0R000101GEO' #Mobile Revenue (Geo) : DTAC
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    ]

mobile_rev_geo_dtac_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
mobile_rev_geo_dtac_df = mobile_rev_geo_dtac_df.loc[mobile_rev_geo_dtac_df['METRIC_CD'].isin(v_metric_list)]

mobile_rev_geo_dtac_df['MOBILE_D'] = np.where(mobile_rev_geo_dtac_df['METRIC_CD']=='DB0R000101GEO', mobile_rev_geo_dtac_df['P'], 0)
mobile_rev_geo_dtac_df['PRE_D'] = np.where(mobile_rev_geo_dtac_df['METRIC_CD']=='DB1R000100', mobile_rev_geo_dtac_df['P'], 0)
mobile_rev_geo_dtac_df['POST_B2C_D'] = np.where(mobile_rev_geo_dtac_df['METRIC_CD']=='DB2R010100', mobile_rev_geo_dtac_df['P'], 0)
mobile_rev_geo_dtac_df['CHK_SUM'] = np.where(mobile_rev_geo_dtac_df['METRIC_CD'].isin(['DB1R000100','DB2R010100']), mobile_rev_geo_dtac_df['P'], 0)

mobile_rev_geo_dtac_df = mobile_rev_geo_dtac_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE_D':'sum', 'PRE_D':'sum', 'POST_B2C_D':'sum', 'CHK_SUM':'sum'})
mobile_rev_geo_dtac_df['CHK_DIFF'] = mobile_rev_geo_dtac_df['MOBILE_D'] - mobile_rev_geo_dtac_df['CHK_SUM']
mobile_rev_geo_dtac_df = mobile_rev_geo_dtac_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_geo_dtac_df = mobile_rev_geo_dtac_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE_D', 'PRE_D', 'POST_B2C_D']]

mod_col_list = mobile_rev_geo_dtac_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_geo_dtac_df[col] = mobile_rev_geo_dtac_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_geo_dtac_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE_D,PRE_D,POST_B2C_D
0,202401,2025-07-17 05:19:06,0,"4,513,086,151","4,513,086,151","1,803,547,883","2,709,538,268"
1,202402,2025-07-17 05:19:06,0,"4,404,498,452","4,404,498,452","1,714,609,789","2,689,888,663"
2,202403,2025-07-17 05:19:06,0,"4,491,153,491","4,491,153,491","1,804,408,125","2,686,745,365"
3,202404,2025-07-17 05:19:06,0,"4,424,562,165","4,424,562,165","1,743,154,862","2,681,407,303"
4,202405,2025-07-17 05:19:06,0,"4,421,410,892","4,421,410,892","1,773,106,846","2,648,304,046"
5,202406,2025-07-17 05:19:06,0,"4,350,271,177","4,350,271,177","1,726,295,968","2,623,975,210"
6,202407,2025-07-17 05:19:06,0,"4,409,326,223","4,409,326,223","1,799,831,136","2,609,495,087"
7,202408,2025-07-17 05:19:06,0,"4,394,834,388","4,394,834,388","1,797,206,658","2,597,627,729"
8,202409,2025-07-17 05:19:06,-0,"4,324,079,486","4,324,079,486","1,737,197,388","2,586,882,098"
9,202410,2025-07-17 05:19:06,0,"4,372,968,515","4,372,968,515","1,782,644,560","2,590,323,955"


## ** Issue History

In [38]:
''' Issue : Postpaid Revenue : TMH (Jan-Jun 2024) '''

v_metict_cd = 'TB2R000100'

issue_postpaid_b2c_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metict_cd]
issue_postpaid_b2c_df = issue_postpaid_b2c_df.loc[issue_postpaid_b2c_df['TM_KEY_MTH']>=202401]
issue_postpaid_b2c_df = issue_postpaid_b2c_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)

issue_postpaid_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202401,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-07-17 02:28:35,"3,722,556,455","3,722,556,455",0,0,0,0
1,202402,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-07-17 02:28:35,"3,686,285,226","3,686,285,226",0,0,0,0
2,202403,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-07-17 02:28:35,"3,658,095,837","3,658,095,837",0,0,0,0
3,202404,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-07-17 02:28:35,"3,692,736,895","3,692,736,895",0,0,0,0
4,202405,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-07-17 02:28:35,"3,675,457,906","3,675,457,906",0,0,0,0
5,202406,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-07-17 02:28:35,"3,712,964,588","3,712,964,588",0,0,0,0
6,202407,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-07-17 02:28:35,"3,667,787,168","3,667,787,168",0,0,0,0
7,202408,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-07-17 02:28:35,"3,680,722,062","3,680,722,062",0,0,0,0
8,202409,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-07-17 02:28:35,"3,706,292,256","3,706,292,256",0,0,0,0
9,202410,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-07-17 02:28:35,"3,685,237,458","3,685,237,458",0,0,0,0


In [39]:
''' Issue : Postpaid Revenue B2C : TMH (Jan-Jun 2024) '''

v_metict_cd = 'TB2R010100'

issue_postpaid_b2c_tmh_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metict_cd]
issue_postpaid_b2c_tmh_df = issue_postpaid_b2c_tmh_df.loc[issue_postpaid_b2c_tmh_df['TM_KEY_MTH']>=202401]
issue_postpaid_b2c_tmh_df = issue_postpaid_b2c_tmh_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)

issue_postpaid_b2c_tmh_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202401,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-07-17 05:19:06,"3,311,282,032","3,311,282,032","3,305,512,181","3,305,485,985","3,305,485,985",0
1,202402,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-07-17 05:19:06,"3,285,072,468","3,285,072,468","3,279,470,827","3,279,449,587","3,279,449,587",0
2,202403,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-07-17 05:19:06,"3,254,097,222","3,254,097,222","3,248,644,556","3,248,622,787","3,248,622,787",0
3,202404,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-07-17 05:19:06,"3,280,178,978","3,280,178,978","3,274,715,195","3,274,699,228","3,274,699,228",0
4,202405,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-07-17 05:19:06,"3,273,314,626","3,273,314,626","3,267,800,934","3,267,780,094","3,267,780,094",0
5,202406,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-07-17 05:19:06,"3,284,957,272","3,284,957,272","3,278,609,892","3,278,595,142","3,278,595,142",0
6,202407,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-07-17 05:19:06,"3,269,232,707","3,269,232,707","3,264,050,341","3,264,050,142","3,264,050,142",0
7,202408,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-07-17 05:19:06,"3,283,314,789","3,283,314,789","3,278,134,554","3,278,134,355","3,278,134,355",0
8,202409,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-07-17 05:19:06,"3,281,480,710","3,281,480,710","3,276,410,628","3,276,410,449","3,276,410,449",0
9,202410,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-07-17 05:19:06,"3,272,186,553","3,272,186,553","3,266,340,594","3,266,340,415","3,266,340,415",0


In [40]:
''' Issue : Postpaid Revenue B2B : TMH (Jan-Jun 2024) '''

v_metict_cd = 'TB2R020100'

issue_postpaid_b2b_tmh_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metict_cd]
issue_postpaid_b2b_tmh_df = issue_postpaid_b2b_tmh_df.loc[issue_postpaid_b2b_tmh_df['TM_KEY_MTH']>=202401]
issue_postpaid_b2b_tmh_df = issue_postpaid_b2b_tmh_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)

issue_postpaid_b2b_tmh_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202401,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-07-17 02:28:35,"411,274,423","411,274,423",0,0,0,0
1,202402,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-07-17 02:28:35,"401,212,758","401,212,758",0,0,0,0
2,202403,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-07-17 02:28:35,"403,998,615","403,998,615",0,0,0,0
3,202404,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-07-17 02:28:35,"412,557,917","412,557,917",0,0,0,0
4,202405,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-07-17 02:28:35,"402,143,280","402,143,280",0,0,0,0
5,202406,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-07-17 02:28:35,"428,007,316","428,007,316",0,0,0,0
6,202407,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-07-17 02:28:35,"398,554,461","398,554,461",0,0,0,0
7,202408,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-07-17 02:28:35,"397,407,274","397,407,274",0,0,0,0
8,202409,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-07-17 02:28:35,"424,811,546","424,811,546",0,0,0,0
9,202410,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-07-17 02:28:35,"413,050,904","413,050,904",0,0,0,0


## ** Current Issue

In [41]:
''' Daily : Postpaid Revenue B2C : TMH '''

v_metict_cd = 'TB2R010100'
# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

issue_daily_post_rev_b2c_tmh_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P', 'G', 'H', 'HH']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
issue_daily_post_rev_b2c_tmh_df = issue_daily_post_rev_b2c_tmh_df.loc[issue_daily_post_rev_b2c_tmh_df['METRIC_CD']==v_metict_cd]
issue_daily_post_rev_b2c_tmh_df = issue_daily_post_rev_b2c_tmh_df.sort_values(by='TM_KEY_DAY', ascending=False).reset_index(drop=True)

issue_daily_post_rev_b2c_tmh_df_display = issue_daily_post_rev_b2c_tmh_df.copy()
mod_col_list = issue_daily_post_rev_b2c_tmh_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    issue_daily_post_rev_b2c_tmh_df_display[col] = issue_daily_post_rev_b2c_tmh_df_display[col].apply(lambda x: format(x, ',.0f'))
issue_daily_post_rev_b2c_tmh_df_display

,TM_KEY_MTH,TM_KEY_DAY,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,P,G,H,HH
0,202507,20250716,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-07-17 05:19:06,nan,nan,nan,nan
1,202507,20250715,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-07-17 05:19:06,nan,nan,nan,nan
2,202507,20250714,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-07-17 05:19:06,nan,nan,nan,nan
3,202507,20250713,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-07-17 05:19:06,"375,592,889","375,592,549","375,592,449","375,592,449"
4,202507,20250712,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-07-17 05:19:06,nan,nan,nan,nan
5,202507,20250711,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-07-17 05:19:06,nan,nan,nan,nan
6,202507,20250710,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-07-17 05:19:06,"442,956,427","442,953,889","442,953,889","442,953,889"
7,202507,20250709,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-07-17 05:19:06,nan,nan,nan,nan
8,202507,20250708,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-07-17 05:19:06,nan,nan,nan,nan
9,202507,20250707,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-07-17 05:19:06,nan,nan,nan,nan
